# Import Packages

In [27]:
import pandas as pd
import numpy as np
import sklearn
import re
import math
import requests as re
import matplotlib.pyplot as plt
from operator import add
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, roc_auc_score, confusion_matrix, accuracy_score
from keras.models import Sequential
from keras import layers
from keras.utils import to_categorical, np_utils
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
import monitoring

In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [ ]:
Data = pd.read_csv("CleanData.csv").drop(columns = ["Unnamed: 0","LyricsOriginal"])
Data["TrackArtist"] = Data.Track +" By "+ Data.Artist1
Data.head()

# Artists Performance

In [ ]:
ArtistAll = Data.Artist2[Data.Artist2!="Nan"].value_counts()[0:10]
ArtistTop10 = Data.Artist2[(Data.Artist2!="Nan") & (Data.Rank <= 10)].value_counts()[0:10]
ArtistTop3 = Data.Artist2[(Data.Artist2!="Nan") & (Data.Rank <= 3)].value_counts()[0:10]
ArtistTop1 = Data.Artist2[(Data.Artist2!="Nan") & (Data.Rank <= 1)].value_counts()[0:10]
ArtistData = pd.DataFrame({
    "Rank" : list(range(1,11)),
    "MostWeeksWithSongsAtTopCharts" : [str(i)+" : "+str(j) for j, i in zip(ArtistTop1.tolist(),pd.DataFrame(ArtistTop1).index.tolist())],
    "MostWeeksWithSongsInTop3Charts" : [str(i)+" : "+str(j) for j, i in zip(ArtistTop3.tolist(),pd.DataFrame(ArtistTop3).index.tolist())],
    "MostWeeksWithSongsInTop10Charts" : [str(i)+" : "+str(j) for j, i in zip(ArtistTop10.tolist(),pd.DataFrame(ArtistTop10).index.tolist())],
    "MostWeeksWithSongsInTop100Charts" : [str(i)+" : "+str(j) for j, i in zip(ArtistAll.tolist(),pd.DataFrame(ArtistAll).index.tolist())]
})
ArtistData.to_csv("Leaderboards/ArtistLeaderboards.csv")
ArtistData.head()

# Songs Performance

In [ ]:
SongsAll = Data.TrackArtist[Data.TrackArtist!="Nan"].value_counts()[0:10]
SongsTop10 = Data.TrackArtist[(Data.TrackArtist!="Nan") & (Data.Rank <= 10)].value_counts()[0:10]
SongsTop3 = Data.TrackArtist[(Data.TrackArtist!="Nan") & (Data.Rank <= 3)].value_counts()[0:10]
SongsTop1 = Data.TrackArtist[(Data.TrackArtist!="Nan") & (Data.Rank <= 1)].value_counts()[0:10]
SongsData = pd.DataFrame({
    "Rank" : list(range(1,11)),
    "MostWeeksAtTopCharts" : [str(i)+" : "+str(j) for j, i in zip(SongsTop1.tolist(),pd.DataFrame(SongsTop1).index.tolist())],
    "MostWeeksInTop3Charts" : [str(i)+" : "+str(j) for j, i in zip(SongsTop3.tolist(),pd.DataFrame(SongsTop3).index.tolist())],
    "MostWeeksInTop10Charts" : [str(i)+" : "+str(j) for j, i in zip(SongsTop10.tolist(),pd.DataFrame(SongsTop10).index.tolist())],
    "MostWeeksInTop100Charts" : [str(i)+" : "+str(j) for j, i in zip(SongsAll.tolist(),pd.DataFrame(SongsAll).index.tolist())]
})
SongsData.to_csv("Leaderboards/SongLeaderboards.csv")
SongsData.head()

# Genre Data

In [ ]:
GenreData = pd.DataFrame({"Date" : Data.Date.unique()})
GenreData["Blues"] = [Data[(Data.Date == i) & (Data.Genre2=="Blues")].shape[0] for i in GenreData.Date]
GenreData["Rock"] = [Data[(Data.Date == i) & (Data.Genre2=="Rock")].shape[0] for i in GenreData.Date]
GenreData["Pop"] = [Data[(Data.Date == i) & (Data.Genre2=="Pop")].shape[0] for i in GenreData.Date]
GenreData["HipHop"] = [Data[(Data.Date == i) & (Data.Genre2=="HipHop")].shape[0] for i in GenreData.Date]
GenreData["R&B"] = [Data[(Data.Date == i) & (Data.Genre2=="R&B")].shape[0] for i in GenreData.Date]
GenreData["Country"] = [Data[(Data.Date == i) & (Data.Genre2=="Country")].shape[0] for i in GenreData.Date]
GenreData["Folk"] = [Data[(Data.Date == i) & (Data.Genre2=="Folk")].shape[0] for i in GenreData.Date]
GenreData["Reggae"] = [Data[(Data.Date == i) & (Data.Genre2=="Reggae")].shape[0] for i in GenreData.Date]
GenreData["Electro"] = [Data[(Data.Date == i) & (Data.Genre2=="Electro")].shape[0] for i in GenreData.Date]
GenreData["Religous"] = [Data[(Data.Date == i) & (Data.Genre2=="Religous")].shape[0] for i in GenreData.Date]
GenreData.head()

# Genre Output

In [ ]:
GenreData.to_csv("GenreData.csv")

# Build Unique Song Dataset For Neural Network

In [ ]:
UniqueData = Data[["TrackArtist","Artist2","Genre1","Genre2","LyricsClean"]].drop_duplicates()
UniqueData = UniqueData[UniqueData.TrackArtist.isna() == False]
UniqueData["MaxRank"] = [max(Data.Rank[Data.TrackArtist == i]) for i in UniqueData.TrackArtist]
UniqueData.head()

In [ ]:
plt.hist(UniqueData.MaxRank)

In [ ]:
for i in UniqueData.Genre2.unique():
    if pd.notna(i):
        fig, axs = plt.subplots(ncols=1, figsize=(15,5))
        sns.distplot(UniqueData.MaxRank[UniqueData.Genre2 == i], ax = axs).set_title("Highest Rank Achieved - {}".format(i))

# Split Training, Test, X & Y

In [ ]:
x = 3
if x >2:
    print("x greater than 2")

In [ ]:
UniqueData = UniqueData.dropna(subset = ["LyricsClean"])

In [ ]:
UniqueData.to_csv("temp.csv")

In [3]:
UniqueData = pd.read_csv("temp.csv").drop(columns = "Unnamed: 0")
UniqueData.head()

,TrackArtist,Artist2,Genre1,Genre2,LyricsClean,MaxRank
0,Poor Little Fool By Ricky Nelson,Ricky Nelson,Pop/Rock,Rock,I used to play around with hearts that hastene...,71.0
1,Patricia By Perez Prado And His Orchestra,Perez Prado,Latin,NaN,Instrumental,85.0
2,Splish Splash By Bobby Darin,Bobby Darin,Pop/Rock,Blues,splish splash I was taking a bath long about a...,3.0
3,Hard Headed Woman By Elvis Presley With The Jo...,Elvis Presley,Pop/Rock,Rock,"Well a hard headed woman, A soft hearted man B...",94.0
4,Rebel-'rouser By Duane Eddy His Twangy Guitar ...,Duane Eddy,Pop/Rock,Rock,Instrumental,52.0


In [4]:
print("Numebr Of Nas For Genre1",len(UniqueData.Genre1)-UniqueData.Genre1.count())
UniqueData.Genre1.value_counts()

Numebr Of Nas For Genre1 343


Pop/Rock         10844
R&B               5073
Country           2184
Rap               1918
Vocal              829
Electronic         494
Jazz               227
Latin              198
Blues              180
Folk               171
Easy               144
Reggae             129
Stage               75
Religious           61
Comedy/Spoken       39
International       27
Children            18
Classical           12
New                  8
Avant                4
Holiday              1
Name: Genre1, dtype: int64

In [5]:
print("Numebr Of Nas For Genre2",len(UniqueData.Genre2)-UniqueData.Genre2.count())
UniqueData.Genre2.value_counts()

Numebr Of Nas For Genre2 4039


Rock        5239
Pop         4090
R&B         3011
Country     2221
HipHop      1871
Blues       1678
Folk         339
Electro      249
Reggae       168
Religous      74
Name: Genre2, dtype: int64

In [6]:
UniqueData = UniqueData.dropna(subset=["Genre1"])
UniqueData = UniqueData[UniqueData.Genre1.isin(["Pop/Rock"
                                               ,"R&B"       
                                               ,"Country"   
                                               ,"Rap"       
                                               ,"Vocal"     
                                               ,"Electronic"
                                               ,"Jazz"      
                                               ,"Latin"     
                                               ,"Blues"     
                                               ,"Folk"      
                                               ,"Easy"      
                                               ,"Reggae"])]
UniqueData.loc[UniqueData.Genre1=="Jazz"] = "Blues"
UniqueData.loc[UniqueData.Genre1=="Vocal"] = "Folk"
XTrain, XTest, YTrain, YTest = train_test_split(UniqueData["LyricsClean"], UniqueData['Genre1'])

# Bag Of Words

In [7]:
Vectorizer = TfidfVectorizer()
Vectorizer.fit(XTrain)
XTrain = Vectorizer.transform(XTrain)
XTest  = Vectorizer.transform(XTest)

# Baseline Logistic Regression Model

In [8]:
Classifier = LogisticRegression(n_jobs=-1)
Classifier.fit(XTrain, YTrain)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
UniqueData.Genre1.unique()

array(['Pop/Rock', 'Latin', 'R&B', 'Country', 'Blues', 'Folk', 'Easy',
       'Reggae', 'Electronic', 'Rap'], dtype=object)

In [10]:
print(f1_score(YTrain,Classifier.predict(XTrain), average="micro"))
print(f1_score(YTest,Classifier.predict(XTest), average="micro"))
confusion_matrix(YTest,Classifier.predict(XTest))

0.6812957780027392
0.631654162200786


array([[  61,    2,    0,    0,    0,    0,   27,    9,    1,    0],
       [   0,   48,    0,    0,    0,    0,  440,   58,    2,    0],
       [   0,    1,    0,    0,    0,    0,   30,    4,    0,    0],
       [   0,    3,    0,    0,    0,    0,   99,   20,    5,    0],
       [   0,    0,    0,    0,  206,    0,   34,    8,    0,    0],
       [   0,    0,    0,    0,    0,    4,   29,    8,    1,    0],
       [   0,   11,    0,    0,    1,    1, 2498,  154,    6,    0],
       [   0,    5,    0,    0,    0,    1,  789,  456,   35,    0],
       [   0,    0,    0,    0,    0,    1,  198,   39,  262,    0],
       [   0,    0,    0,    0,    0,    0,   31,    4,    5,    1]])

# Baseline Deep Learning Model - Sequential Neural Network

In [11]:
Encoder = LabelEncoder()
Encoder.fit(YTrain)
YTrainNum = Encoder.transform(YTrain)
YTrainDummy = np_utils.to_categorical(YTrainNum)
YTestNum = Encoder.transform(YTest)
YTestDummy = np_utils.to_categorical(YTestNum)

In [12]:
InputDim = XTrain.shape[1]
Model = Sequential()
Model.add(layers.Dense(30, input_dim=InputDim, activation='relu'))
Model.add(layers.Dropout(rate = 0.4))
Model.add(layers.Dense(20, activation='relu'))
Model.add(layers.Dropout(rate = 0.4))
Model.add(layers.Dense(10, activation='softmax'))

Model.compile(loss='categorical_crossentropy', 
              optimizer="adam",
              metrics=["acc"])
Model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 30)                1157100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                620       
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
Total params: 1,157,930
Trainable params: 1,157,930
Non-trainable params: 0
___________

In [13]:
Model.fit(XTrain, YTrainDummy,
          epochs=25,
          validation_data=(XTest, YTestDummy),
          batch_size=15,
          verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 16793 samples, validate on 5598 samples
Epoch 1/25
16793/16793 [==============================] - 9s 559us/step - loss: 1.4883 - acc: 0.5073 - val_loss: 1.2450 - val_acc: 0.5688
Epoch 2/25
16793/16793 [==============================] - 8s 449us/step - loss: 1.2006 - acc: 0.5779 - val_loss: 1.1316 - val_acc: 0.6008
Epoch 3/25
16793/16793 [==============================] - 8s 448us/step - loss: 1.0629 - acc: 0.6237 - val_loss: 1.0803 - val_acc: 0.6440
Epoch 4/25
16793/16793 [==============================] - 8s 449us/step - loss: 0.9453 - acc: 0.6750 - val_loss: 1.0656 - val_acc: 0.6672
Epoch 5/25
16793/16793 [==============================] - 8s 448us/step - loss: 0.8369 - acc: 0.7172 - val_loss: 1.0787 - val_acc: 0.6777
Epoch 6/25
16793/16793 [==============================] - 8s 447us/step - loss: 0.7655 - acc: 0.7426 - val_loss: 1.1071 - val_acc: 0.6793
Epoch 7/25
16793/16793 [==============================] - 8s 447us/step - l

In [14]:
print(f1_score(YTrainNum,Model.predict_classes(XTrain), average="micro"))
print(f1_score(YTestNum,Model.predict_classes(XTest), average="micro"))
confusion_matrix(YTestNum,Model.predict_classes(XTest))

0.9021616149586137
0.6947123972847445


array([[  69,    1,    0,    1,    0,    0,   17,   12,    0,    0],
       [   0,  253,    0,    7,    0,    0,  198,   85,    5,    0],
       [   0,    0,    0,   10,    0,    0,   20,    4,    1,    0],
       [   0,    4,    0,   46,    0,    0,   48,   26,    3,    0],
       [   0,    2,    0,    2,  212,    0,   25,    6,    1,    0],
       [   0,    7,    0,    8,    0,    4,   16,    6,    1,    0],
       [   6,  106,    0,   20,    4,    0, 2132,  367,   36,    0],
       [   0,   38,    0,   21,    1,    1,  350,  833,   42,    0],
       [   1,    5,    0,    5,    0,    0,   74,   75,  340,    0],
       [   0,    5,    0,   16,    0,    1,   10,    6,    3,    0]])

# Word Embedding Neural Network

In [15]:
XTrain, XTest, YTrain, YTest = train_test_split(UniqueData["LyricsClean"], UniqueData['Genre1'])

In [16]:
Encoder = LabelEncoder()
Encoder.fit(YTrain)
YTrainNum = Encoder.transform(YTrain)
YTrainDummy = np_utils.to_categorical(YTrainNum)
YTestNum = Encoder.transform(YTest)
YTestDummy = np_utils.to_categorical(YTestNum)

In [17]:
TokenModel = Tokenizer(num_words=5000)
TokenModel.fit_on_texts(XTrain.tolist())
XTrain = TokenModel.texts_to_sequences(XTrain.tolist())
#XTrain = TokenModel.texts_to_matrix(XTrain, mode='tfidf')
XTest = TokenModel.texts_to_sequences(XTest.tolist())
#XTest = TokenModel.texts_to_matrix(XTest, mode='tfidf')

In [18]:
XTrain = pad_sequences(XTrain, padding='post', maxlen=500)
XTest = pad_sequences(XTest, padding='post', maxlen=500)

In [19]:
InputDim = XTrain.shape[1]
Model = Sequential()
Model.add(layers.Embedding(input_dim=5000,
                           output_dim=200,
                           input_length=500))
Model.add(layers.GlobalMaxPooling1D())
#Model.add(layers.Flatten())
Model.add(layers.Dense(50, activation='relu'))
#Model.add(layers.SpatialDropout1D(0.2))
#Model.add(layers.GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
#Model.add(layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#Model.add(layers.Dense(40, activation='relu'))
#Model.add(layers.Dropout(rate = 0.3))
Model.add(layers.Dense(10, activation='softmax'))

Model.compile(loss='categorical_crossentropy', 
              optimizer="adam",
              metrics=["acc"])
Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 200)          1000000   
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                10050     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                510       
Total params: 1,010,560
Trainable params: 1,010,560
Non-trainable params: 0
_________________________________________________________________


In [20]:
Model.fit(XTrain, 
          YTrainDummy,
          epochs=25,
          validation_data=(XTest, YTestDummy),
          batch_size=15,
          verbose=1)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 16793 samples, validate on 5598 samples
Epoch 1/25
16793/16793 [==============================] - 6s 359us/step - loss: 1.2840 - acc: 0.5635 - val_loss: 1.1574 - val_acc: 0.5990
Epoch 2/25
16793/16793 [==============================] - 6s 332us/step - loss: 1.0848 - acc: 0.6388 - val_loss: 1.0808 - val_acc: 0.6377
Epoch 3/25
16793/16793 [==============================] - 6s 332us/step - loss: 0.9311 - acc: 0.6964 - val_loss: 1.0557 - val_acc: 0.6538
Epoch 4/25
16793/16793 [==============================] - 6s 332us/step - loss: 0.7916 - acc: 0.7474 - val_loss: 1.0462 - val_acc: 0.6738
Epoch 5/25
16793/16793 [==============================] - 6s 332us/step - loss: 0.6680 - acc: 0.7913 - val_loss: 1.0587 - val_acc: 0.6729
Epoch 6/25
16793/16793 [==============================] - 6s 332us/step - loss: 0.5744 - acc: 0.8163 - val_loss: 1.0763 - val_acc: 0.6897
Epoch 7/25
16793/16793 [=====================

In [21]:
print(f1_score(YTrainNum,Model.predict_classes(XTrain), average="micro"))
print(f1_score(YTestNum,Model.predict_classes(XTest), average="micro"))
print("😊")
confusion_matrix(YTestNum,Model.predict_classes(XTest))

0.9241350562734473
0.7031082529474812
😊


array([[  67,    2,    0,    0,    1,    0,   21,    9,    0,    2],
       [   0,  238,    2,    3,    3,    2,  184,   81,    9,    2],
       [   0,    1,    6,    0,    1,    0,   16,    8,    0,    0],
       [   0,    7,    0,   41,    0,    1,   61,   26,    5,    0],
       [   0,    5,    0,    0,  229,    0,   15,    9,    0,    0],
       [   0,    1,    0,    0,    0,   25,    9,   11,    0,    0],
       [   2,  105,    1,    3,    8,   14, 2090,  399,   42,    7],
       [   1,   56,    3,    7,    4,    6,  332,  897,   24,    2],
       [   1,    2,    0,    3,    0,    0,   80,   53,  324,    1],
       [   0,    0,    0,    0,    0,    0,    5,    2,    2,   19]])

# LTSTM Neural Network

In [22]:
XTrain, XTest, YTrain, YTest = train_test_split(UniqueData["LyricsClean"], UniqueData['Genre1'])

In [23]:
Encoder = LabelEncoder()
Encoder.fit(YTrain)
YTrainNum = Encoder.transform(YTrain)
YTrainDummy = np_utils.to_categorical(YTrainNum)
YTestNum = Encoder.transform(YTest)
YTestDummy = np_utils.to_categorical(YTestNum)

In [24]:
TokenModel = Tokenizer(num_words=5000)
TokenModel.fit_on_texts(XTrain.tolist())
XTrain = TokenModel.texts_to_sequences(XTrain.tolist())
#XTrain = TokenModel.texts_to_matrix(XTrain, mode='tfidf')
XTest = TokenModel.texts_to_sequences(XTest.tolist())
#XTest = TokenModel.texts_to_matrix(XTest, mode='tfidf')

In [25]:
XTrain = pad_sequences(XTrain, padding='post', maxlen=500)
XTest = pad_sequences(XTest, padding='post', maxlen=500)

In [32]:
InputDim = XTrain.shape[1]
Model = Sequential()
Model.add(layers.Embedding(input_dim=5000,
                           output_dim=100,
                           input_length=500))
#Model.add(layers.GlobalMaxPooling1D())
#Model.add(layers.Flatten())
#Model.add(layers.Dense(50, activation='relu'))
Model.add(layers.SpatialDropout1D(0.2))
#Model.add(layers.GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
Model.add(layers.CuDNNLSTM(100))
#Model.add(layers.Dense(40, activation='relu'))
Model.add(layers.Dropout(rate = 0.3))
Model.add(layers.Dense(10, activation='softmax'))

Model.compile(loss='categorical_crossentropy', 
              optimizer="adam",
              metrics=["acc"])
Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 100)          500000    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 500, 100)          0         
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 100)               80800     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1010      
Total params: 581,810
Trainable params: 581,810
Non-trainable params: 0
_________________________________________________________________


In [33]:
Model.fit(XTrain, 
          YTrainDummy,
          epochs=100,
          validation_data=(XTest, YTestDummy),
          batch_size=128,
          verbose=1)

Train on 16793 samples, validate on 5598 samples
Epoch 1/100
16793/16793 [==============================] - 9s 550us/step - loss: 1.5762 - acc: 0.4842 - val_loss: 1.4442 - val_acc: 0.5211
Epoch 2/100
16793/16793 [==============================] - 8s 478us/step - loss: 1.4713 - acc: 0.5111 - val_loss: 1.4249 - val_acc: 0.5263
Epoch 3/100
16793/16793 [==============================] - 8s 480us/step - loss: 1.4567 - acc: 0.5165 - val_loss: 1.4189 - val_acc: 0.5280
Epoch 4/100
16793/16793 [==============================] - 8s 480us/step - loss: 1.4467 - acc: 0.5180 - val_loss: 1.4307 - val_acc: 0.5266
Epoch 5/100
16793/16793 [==============================] - 8s 478us/step - loss: 1.4352 - acc: 0.5209 - val_loss: 1.4074 - val_acc: 0.5304
Epoch 6/100
16793/16793 [==============================] - 8s 480us/step - loss: 1.4244 - acc: 0.5216 - val_loss: 1.3990 - val_acc: 0.5302
Epoch 7/100
16793/16793 [==============================] - 8s 481us/step - loss: 1.4039 - acc: 0.5243 - val_loss: 1.3

16793/16793 [==============================] - 8s 478us/step - loss: 0.7980 - acc: 0.7681 - val_loss: 1.2024 - val_acc: 0.6345
Epoch 60/100
16793/16793 [==============================] - 8s 479us/step - loss: 0.7834 - acc: 0.7740 - val_loss: 1.2075 - val_acc: 0.6195
Epoch 61/100
16793/16793 [==============================] - 8s 480us/step - loss: 0.7643 - acc: 0.7794 - val_loss: 1.2139 - val_acc: 0.6347
Epoch 62/100
16793/16793 [==============================] - 8s 478us/step - loss: 0.7531 - acc: 0.7821 - val_loss: 1.1917 - val_acc: 0.6363
Epoch 63/100
16793/16793 [==============================] - 8s 480us/step - loss: 0.7319 - acc: 0.7876 - val_loss: 1.2222 - val_acc: 0.6486
Epoch 64/100
16793/16793 [==============================] - 8s 480us/step - loss: 0.7223 - acc: 0.7910 - val_loss: 1.2564 - val_acc: 0.6467
Epoch 65/100
16793/16793 [==============================] - 8s 479us/step - loss: 0.7069 - acc: 0.7944 - val_loss: 1.2577 - val_acc: 0.6199
Epoch 66/100
16793/16793 [=======

In [34]:
print(f1_score(YTrainNum,Model.predict_classes(XTrain), average="micro"))
print(f1_score(YTestNum,Model.predict_classes(XTest), average="micro"))
print("😊")
confusion_matrix(YTestNum,Model.predict_classes(XTest))

0.7704996129339606
0.6382636655948553
😊


array([[   0,   11,    0,    0,   53,    0,   31,    9,    2,    0],
       [   0,  119,    0,    0,   12,    1,  347,   47,    4,    0],
       [   0,    3,    0,    0,    1,    1,   27,    2,    0,    0],
       [   0,    5,    0,    1,    2,    4,   75,   29,    1,    0],
       [   0,    7,    0,    0,  212,    0,   31,    5,    0,    0],
       [   0,    2,    0,    3,    1,    6,   25,   10,    0,    0],
       [   0,  102,    0,    5,   12,    6, 2362,  255,   20,    3],
       [   0,   43,    0,    3,    7,    6,  560,  612,   18,    2],
       [   0,    9,    0,    8,    2,    5,  116,   64,  259,    6],
       [   0,    1,    0,    0,    0,    2,   11,    6,    2,    2]])